In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
def predict_step(img_path):
  raw_image = Image.open(img_path)
  inputs = processor(raw_image, return_tensors="pt").to("cuda")
  out = model.generate(**inputs)
  return processor.decode(out[0], skip_special_tokens=True)

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [2]:
import re

def extract_twitter_urls(text):
    pattern = r'https://pbs\.twimg\.com/media/[A-Za-z0-9_-]+\?format=jpg&name=small'
    matches = re.findall(pattern, text)
    return matches[0] if matches else None


In [3]:
import requests

def download_and_save_image(url, destination_path):
      response = requests.get(url, stream=True)
      if response.status_code == 200:
          with open(destination_path, 'wb') as file:
              for chunk in response.iter_content(chunk_size=128):
                  file.write(chunk)
      else :
        return

In [ ]:
folder_path=''
for i in range(len(df['media'])):
  url=extract_twitter_urls(df['media'][i])
  if url==None:
    continue
  download_and_save_image(url,folder_path+str(i)+'.jpg')

In [ ]:
def extract_integers_from_string(file_name):
    integers = re.findall(r'\d+', file_name)
    integers = [int(num) for num in integers]
    return integers[0]

In [ ]:
df['captions']=pd.Series(dtype='str')

In [ ]:
directory2 = 'directory_path'
for filename in os.listdir(directory2):
  f=os.path.join(directory2, filename)
  idx=extract_integers_from_string(filename)
  df['captions'][idx]=predict_step(f)

In [ ]:
for i in range(len(df)):
  if type(df['captions'][i])==float:
    df['captions'][i]=''

In [ ]:
import gpt_2_simple
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
my_model = gpt_2_simple.from_pretrained(model_path)

In [ ]:
df['content']=pd.Series(dtype='str')

In [ ]:
for i in range(len(df)):
  prompt=prompt = 'date: '+df['date'][i]+' likes: '+df['likes'][i]+' caption: '+predict_step(image_path)
  encoded_results = tokenizer(prompt, return_tensors='pt')
  tok_ids = encoded_results['input_ids']
  gen_ids = my_model.generate(tok_ids)
  decoded_str = tokenizer.decode(gen_ids)
  pattern = r'tweets:(.*)'
  match = re.search(pattern, decoded_str)
  if match:
      extracted_text = match.group(1).strip()
  else:
      df['content'][i]=' '